In [ ]:
# Install dependencies for pulling OSM data
!pip install leafmap geopandas osmnx shapely
import leafmap
import osmnx
import geopandas
import shapely

In [ ]:
# Create a map over Parkway West
m = leafmap.Map(center=[38.622800689786146, -90.53972968259362], zoom=15)
m

In [ ]:
# Let's add some satellite imagery!
m = leafmap.Map(center=[38.622800689786146, -90.53972968259362], zoom=15)
m.add_tile_layer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    name="Google Satellite",
    attribution="Google",
)
m

In [ ]:
# Chris is hungry... Let's make a new map and find some restaraunts
restaraunt_map = leafmap.Map(center=[38.49481428709007, -90.29638904026217], zoom=15)
restaraunt_map.add_tile_layer(
    url="https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
    name="Google Satellite",
    attribution="Google",
)
restaraunt_map

In [ ]:
# Let's query the data using GeoPandas Data Frames
north, south, east, west = 38.544406, 38.485307, -90.270538, -90.365467
gdf = leafmap.osm_gdf_from_bbox(
    north, south, east, west, 
    tags={"building": ['yes', 'retail', 'house']})
gdf

In [ ]:
# Convert the projection system from geographic to projected
gdf = gdf.to_crs(epsg=3395)  
gdf

In [ ]:
# Add all buildings which met query criteria above to map
restaraunt_map.add_gdf(gdf, layer_name="All Buildings", fill_colors=["green"])
restaraunt_map

In [ ]:
# Trim data frames to only contain two columns
small_gdf = gdf[["name", "addr:street", "amenity", "geometry"]]

# Make a new dataframe of only restaraunts
restaraunts = small_gdf.query("amenity in ['restaraunt', 'fast_food']")
restaraunts

In [ ]:
# Convert to points
restaraunts['geometry'] = restaraunts['geometry'].centroid
restaraunts

In [ ]:
# Add it to the map!
restaraunt_map.add_gdf(restaraunts, layer_name="Restaraunts", fill_colors=["green"])
restaraunt_map

In [ ]:
#specific_feature = small_gdf[small_gdf['name'] == 'Walmart Supercenter'].iloc[0]
# Create a data frame of just Walmart
walmart = small_gdf[gdf['name'] == 'Walmart Supercenter']
walmart


In [ ]:
# Buffer the Walmart by 1-mile (~1600m)
walmart['geometry'] = walmart['geometry'].buffer(1600)
buffered_geometry = walmart['geometry']
# Ignore the warning for now...

In [ ]:
# Add that buffer to the map!
restaraunt_map.add_gdf(walmart, layer_name="Walmart Buffer", fill_colors=["red"])
restaraunt_map

In [ ]:
restaraunts = restaraunts.reset_index(drop=True)
walmart = walmart.reset_index(drop=True)

buildings_within_range = restaraunts[restaraunts.intersects(walmart['geometry'])]

In [ ]:
restaurants_within_range = geopandas.sjoin(restaraunts, walmart, how="inner", op="within")

# Convert to points
restaurants_within_range['geometry'] = restaurants_within_range['geometry'].centroid

# Drop columns from the right-join Walmart table
columns_to_drop = [col for col in restaurants_within_range.columns if col.endswith("_right")]
restaurants_within_range = restaurants_within_range.drop(columns=columns_to_drop)


restaurants_within_range

In [ ]:
# Add the restaraunts within 1-mile to the map!
restaraunt_map.add_gdf(restaurants_within_range, layer_name="Restaraunt Choices", fill_colors=["green"])
restaraunt_map